In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from src.utils.trust_region import TrustRegion
from src.utils.lagrange_polynomial import LagrangePolynomials
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm
import numpy as np
import casadi as ca

# def myfunc(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
#     return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100
def myfunc(x:np.ndarray) -> np.ndarray:    
    return 10*(x[0]**2)*(1 + 0.75*np.cos(70*x[0])/12) + np.cos(100*x[0])**2/24 + 2*(x[1]**2)*(1 + 0.75*np.cos(70*x[1])/12) + np.cos(100*x[1])**2/24 + 4*x[0]*x[1]

tr_list = []
initial_centers = [np.array([[2.2], [2.12]]), 
                np.array([[-2.2], [-2.12]]),
                np.array([[-2.2], [2.12]]),
                np.array([[2.2], [-2.12]])]

for center in initial_centers:
    dataset = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [1.0, 1.0], [0.0, -1.0]]).T + center
    INPUT_SYMBOL = ca.SX.sym('x', dataset.shape[0])

    poly_mf = LagrangePolynomials(input_symbols=INPUT_SYMBOL, pdegree=2)
    poly_mceq = LagrangePolynomials(input_symbols=INPUT_SYMBOL, pdegree=2)
    mf = []
    for i in range(dataset.shape[1]):
        x = dataset[:, i]
        mf.append(myfunc(x))
    mf = np.array(mf)
    
    sort_ind = mf.argsort(axis=0)  
    dataset = dataset[:, sort_ind]
    mf = mf[sort_ind]

    tr = TrustRegion(input_symbols=INPUT_SYMBOL, dataset=dataset, results=mf, func=myfunc)
    tr.run(max_radius=1.5, max_iter=30, rad_tol=1E-8)
    
    tr_list.append(tr)

/home/iffanh/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


====================Iteration 0====================

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  14.00us (736.84ns)  14.39us (757.32ns)        19
       nlp_g  |  18.00us (947.37ns)  16.63us (875.47ns)        19
  nlp_grad_f  |  15.00us (833.33ns)  15.65us (869.28ns)        18
  nlp_hess_l  |  13.00us (812.50ns)  13.77us (860.44ns)        16
   nlp_jac_g  |  11.00us (611.11ns)  11.07us (615.11ns)        18
       total  |   3.41ms (  3.41ms)   6.80ms (  6.80ms)         1
point from step calculation = [0.2165182  0.13384893]
Best point = [0.2165182  0.13384893]
Best OF

Best point = [ 0.01459353 -0.02069185]
Best OF: 0.01189009820067346
rho : 0.077069
Radius: 0.375
All points : [[ 1.45935319e-02  1.12780151e-04  5.44172276e-02  1.47776257e-02
  -3.56537462e-01  7.58989825e-01]
 [-2.06918463e-02  3.95104061e-02 -2.13657626e-01  5.03877921e-01
   6.99283563e-01 -1.12202362e-01]]
====================Iteration 13====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  12.00us (  1.00us)   8.54us (711.67ns)        12
       nlp_g  |  12.00us (  1.00us)  12.48us (  1.04us)        12
  nlp_grad_f  |  11.00us (916.67ns)  10.61us (883.92ns)        12
  nlp_hess_l  |   9.00us (900.00ns)   8.36us (835.90ns)        10
   nlp_jac_g  |   9.00us (750.00ns)   7.33us (611.08ns)        12
       total  |   1.96ms (  1.96ms)   1.96ms (  1.96ms)         1
point from step calculation = [0.01050382 0.02072068]
Best point = [ 0.01459353 -0.02069185]
Best OF: 0.01189009820067346
rho : 0.91505
Radius: 0.375
All points : [[ 0.0145

Could not construct polynomials with poisedness < 1.2 after 10 iterations. Consider increasing the max_iter.
Best point = [ 0.01459353 -0.02069185]
Best OF: 0.01189009820067346
rho : 0.103077
Radius: 0.007031249999999999
All points : [[ 0.01459353  0.02664606  0.01368356  0.00170707  0.00110643  0.02891296]
 [-0.02069185 -0.01344679 -0.04671755  0.00430725 -0.02467333 -0.04244316]]
====================Iteration 24====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  14.00us (700.00ns)  14.04us (702.00ns)        20
       nlp_g  |  21.00us (  1.05us)  19.29us (964.25ns)        20
  nlp_grad_f  |  15.00us (789.47ns)  17.13us (901.37ns)        19
  nlp_hess_l  |  15.00us (882.35ns)  14.44us (849.24ns)        17
   nlp_jac_g  |  12.00us (631.58ns)  11.07us (582.58ns)        19
       total  |   3.04ms (  3.04ms)   3.03ms (  3.03ms)         1
point from step calculation = [ 0.01515763 -0.02770043]
Best point = [ 0.01459353 -0.02069185]
Best 

Best point = [ 0.0273672  -0.00319162]
Best OF: 0.07976982254253759
rho : 0.993829
Radius: 1.5
All points : [[-0.2165182   0.24393532 -1.2        -2.2        -2.2        -2.2       ]
 [-0.13384892 -0.5204665  -1.12       -1.12       -2.12       -3.12      ]]
====================Iteration 3====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   7.00us (636.36ns)  14.29us (  1.30us)        11
       nlp_g  |   9.00us (818.18ns)   9.53us (866.36ns)        11
  nlp_grad_f  |  10.00us (909.09ns)  10.03us (911.36ns)        11
  nlp_hess_l  |   8.00us (888.89ns)   8.33us (925.33ns)         9
   nlp_jac_g  |   6.00us (545.45ns)   6.58us (598.18ns)        11
       total  |   1.75ms (  1.75ms)   1.74ms (  1.74ms)         1
point from step calculation = [-0.04518149  0.13723673]
Best point = [-0.04518149  0.13723673]
Best OF: 0.037967808179260015
rho : 1.00036
Radius: 1.5
All points : [[ 0.0273672  -0.2165182   0.24393532 -1.2        -2.2        -

...

Could not construct polynomials with poisedness < 1.2 after 10 iterations. Consider increasing the max_iter.
Best point = [-0.01250664  0.0243257 ]
Best OF: 0.0297131923876376
rho : -0.110367
Radius: 0.44999999999999996
All points : [[-0.01250664  0.03233951 -0.02839051 -0.09102133 -0.51953306  1.36629572]
 [ 0.0243257   0.01639592  0.05516364 -0.17242616  1.41934641  1.48376248]]
====================Iteration 9====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   9.00us (818.18ns)   8.11us (737.55ns)        11
       nlp_g  |  16.00us (  1.45us)  13.05us (  1.19us)        11
  nlp_grad_f  |  12.00us (  1.00us)  10.88us (906.50ns)        12
  nlp_hess_l  |   9.00us (900.00ns)   9.16us (915.80ns)        10
   nlp_jac_g  |   9.00us (750.00ns)   7.36us (613.17ns)        12
       total  |   2.12ms (  2.12ms)   2.11ms (  2.11ms)         1
point from step calculation = [ 0.14446614 -0.31329039]
Best point = [-0.01250664  0.0243257 ]
Best OF

Could not construct polynomials with poisedness < 1.2 after 10 iterations. Consider increasing the max_iter.
Best point = [-0.01250664  0.0243257 ]
Best OF: 0.0297131923876376
rho : 0.393686
Radius: 0.04859999999999999
All points : [[-0.01250664  0.0017618  -0.07686146 -0.04790712 -0.10959495  0.12855586]
 [ 0.0243257  -0.01002863  0.17114565 -0.13375911  0.01966728 -0.05182556]]
====================Iteration 21====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   7.00us (700.00ns)   7.15us (715.20ns)        10
       nlp_g  |  10.00us (  1.00us)   9.94us (994.30ns)        10
  nlp_grad_f  |  11.00us (  1.00us)   9.53us (866.18ns)        11
  nlp_hess_l  |   9.00us (  1.00us)   8.05us (894.22ns)         9
   nlp_jac_g  |   6.00us (545.45ns)   6.56us (596.36ns)        11
       total  |   1.68ms (  1.68ms)   1.67ms (  1.67ms)         1
point from step calculation = [0.01759099 0.06248449]
Could not construct polynomials with poisedness 

Best point = [0.14938317 0.10766954]
Best OF: 0.32760058129168546
rho : 1.01564
Radius: 1.5
All points : [[ 0.1356259   0.35444601  2.2         2.2         3.2         3.2       ]
 [ 0.20964778 -0.61325627 -2.12       -3.12       -2.12       -1.12      ]]
====================Iteration 3====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  11.00us (733.33ns)  10.63us (709.00ns)        15
       nlp_g  |  16.00us (  1.07us)  14.66us (977.40ns)        15
  nlp_grad_f  |  15.00us (  1.07us)  12.33us (880.50ns)        14
  nlp_hess_l  |  11.00us (916.67ns)  11.30us (941.67ns)        12
   nlp_jac_g  |   8.00us (571.43ns)   8.54us (610.00ns)        14
       total  |   2.37ms (  2.37ms)   2.37ms (  2.37ms)         1
point from step calculation = [ 0.15013944 -0.02093201]
Best point = [ 0.15013944 -0.02093201]
Best OF: 0.24214450451365643
rho : 1.04375
Radius: 1.5
All points : [[ 0.14938317  0.1356259   2.2         2.2         3.2         3.2 

Best point = [-0.0225996   0.05705698]
Best OF: 0.052286206996421225
rho : 0.871067
Radius: 0.75
All points : [[-0.0225996  -0.03720093  0.04112247  0.09286637 -0.15243004  0.98677624]
 [ 0.05705698  0.0851929   0.06401177  0.24331243  0.41078471 -0.85350382]]
====================Iteration 13====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   9.00us (  1.00us)   6.97us (774.22ns)         9
       nlp_g  |  12.00us (  1.33us)  10.28us (  1.14us)         9
  nlp_grad_f  |  12.00us (  1.20us)   9.34us (934.00ns)        10
  nlp_hess_l  |   7.00us (875.00ns)   7.12us (890.00ns)         8
   nlp_jac_g  |   8.00us (800.00ns)   6.25us (625.00ns)        10
       total  |   1.65ms (  1.65ms)   1.64ms (  1.64ms)         1
point from step calculation = [-0.06768911  0.80570038]
Best point = [-0.0225996   0.05705698]
Best OF: 0.052286206996421225
rho : 0.278209
Radius: 0.375
All points : [[-2.25995969e-02 -3.72009266e-02 -5.24011936e-03  4.1122

Best point = [0.01333049 0.01449894]
Best OF: 0.005967777301696
rho : -0.576437
Radius: 0.04859999999999999
All points : [[ 0.01333049 -0.04719435  0.03918845  0.04376243 -0.05515204 -0.08273982]
 [ 0.01449894  0.05205222 -0.08499526  0.0863262   0.14784357  0.00662278]]
====================Iteration 26====================
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   9.00us (818.18ns)   7.83us (712.09ns)        11
       nlp_g  |  12.00us (  1.09us)  10.74us (976.00ns)        11
  nlp_grad_f  |  11.00us (916.67ns)  10.32us (860.17ns)        12
  nlp_hess_l  |   9.00us (900.00ns)   9.19us (919.10ns)        10
   nlp_jac_g  |   9.00us (750.00ns)   7.41us (617.75ns)        12
       total  |   1.85ms (  1.85ms)   1.85ms (  1.85ms)         1
point from step calculation = [-0.00169109  0.01362623]
Best point = [0.01333049 0.01449894]
Best OF: 0.005967777301696
rho : 0.207471
Radius: 0.024299999999999995
All points : [[ 0.01333049  0.07614036  0.0300973

In [3]:
fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0

style = 'default'

plt.style.use(style)
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

fig, ax = plt.subplots(1)
for i, tr in enumerate(tr_list):
    ## Plot of objective function
    ax.plot(range(len(tr.list_of_OF)), tr.list_of_OF, label=f"dataset-{i+1}", linestyle='dashed')
    try:
        ax.set_yscale("log")
    except:
        ax.set_yscale("linear")
    ax.set_xlabel(f"Number of iterations")
    ax.set_ylabel(f"OF value")

plt.legend()
plt.savefig(f"./plots/score_OF_all.png")
plt.close()

In [4]:
asdf

NameError: name 'asdf' is not defined

In [6]:

from casadi import *
import glob

x, y = np.meshgrid(np.linspace(-4, 4, 100),
                    np.linspace(-4, 4, 100))
    
# levels = list(np.arange(0, 5, 1))
# levels = [0.01, 0.05, 0.1, 0.5, 1, 5, 10]
# levels = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50]
levels = [10**i for i in np.arange(-1,2.5,0.2)]

old_center = [0,0]
old_radius = 0
    
files = glob.glob('./plots/*')
for f in files:
    os.remove(f)
tr = tr_list[1]
        
for i, m in enumerate(tr.list_of_models):
    if i % 1 == 0:
        center = m.sample_set.ball.center
        radius = tr.list_of_radius[i]
        intx = x - center[0]
        inty = y - center[1]
        dist = np.sqrt(intx**2 + inty**2)
        intindices = dist <= radius

        fig, ax = plt.subplots(1)
#         ax.set_title(f"{tr.list_of_status[i]['criticality_step']} \n {tr.list_of_status[i]['acceptance']}")
        ax.set_xlabel(f"$x_1$")
        ax.set_ylabel(f"$x_2$")
        func = myfunc([x, y])
        func_map = m.model_polynomial.feval.map(x.shape[0]*x.shape[1])
        x_dm = DM(x).reshape((1, x.shape[0]*x.shape[1]))
        y_dm = DM(y).reshape((1, x.shape[0]*x.shape[1]))
        # print(func_map, vertcat(x_dm, y_dm).shape)
        func_dm = func_map(vertcat(x_dm, y_dm))
        func_dm = func_dm.reshape((x.shape[0], x.shape[1])).full()
        func[intindices] = func_dm[intindices]
        
        
        circle1 = plt.Circle(center, radius, color='black', fill=False)
        circle2 = plt.Circle(old_center, old_radius, color='red', fill=False)
        CS = ax.contour(x, y, func, levels, norm = LogNorm(), cmap=cm.PuBu_r, locator=ticker.LogLocator())
        ax.clabel(CS, fontsize=9, inline=True)
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        plt.scatter(m.y[0, :], m.y[1, :], color='black', label=f'iteration_{i}')
        plt.scatter(m.y[0, 0], m.y[1, 0], color='red', label=f'Best point')
        plt.legend()
        plt.savefig(f"./plots/TR_plots_{i}.png")
        plt.close()

        old_center = center*1
        old_radius = radius*1

        
## Plot of objective function
fig, ax = plt.subplots(1)
ax.set_title(f"Objective Function vs Iterations")
ax.scatter(range(len(tr.list_of_OF)), tr.list_of_OF, color="black", label="OF")
try:
    ax.set_yscale("log")
except:
    ax.set_yscale("linear")

plt.legend()
plt.savefig(f"./plots/score_OF.png")
plt.close()

/tmp/ipykernel_727/685025521.py:47: UserWarning: Log scale: values of z <= 0 have been masked
  CS = ax.contour(x, y, func, levels, norm = LogNorm(), cmap=cm.PuBu_r, locator=ticker.LogLocator())
